# Loading NYU Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset

import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np

import timeit
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
print(os.path.isfile('../bedrooms_part3/bedroom_0041/bedroom_0041_x.mat'))

False


In [2]:
import os
class DepthDataset(Dataset):
    def __init__(self, data_dir):
        self.data_files=[]
        folders=os.listdir(data_dir+'/RGB')
        for folder in folders:
            subfolders=os.listdir(data_dir+'/RGB/'+folder)
            for subfolder in subfolders:
                if subfolder.startswith('.'):
                    continue
                files=os.listdir(data_dir+'/RGB/'+folder+'/'+subfolder)
                for file in files:
                    if file.endswith('.mat'):
                        
                        self.data_files.append(folder+'/'+ subfolder+'/'+file)
            self.data_dir=data_dir
#         sort(self.data_files)      

    def __getitem__(self, index):
        name=self.data_files[index]
        img=torch.from_numpy(h5py.File(self.data_dir+'/RGB/'+name,'r')['rgbOut'].value).float()
        depth=torch.from_numpy(h5py.File(self.data_dir+'/DEP/'+name,'r')['depthOut'].value).float()
        return img,depth

    def __len__(self):
        return len(self.data_files)

In [3]:
dataset=DepthDataset('../BiggerData')
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter((torch.randperm(self.num_samples)+self.start).long())

    def __len__(self):
        return self.num_samples

In [4]:
N=dataset.__len__()
NUM_TRAIN = int(N*0.9)
NUM_VAL = N-NUM_TRAIN
print("NUM_TRAIN:",NUM_TRAIN,",NUM_VAL:",NUM_VAL)
batch_size=6
loader_train = DataLoader(dataset, batch_size=batch_size, sampler=ChunkSampler(NUM_TRAIN, 0),num_workers=8)
loader_val = DataLoader(dataset, batch_size=batch_size, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN),num_workers=8)

NUM_TRAIN: 15257 ,NUM_VAL: 1696


In [5]:
# image size = [304, 228]
# depth size = [74,55]
print(len(loader_train))

2543


In [8]:
for t,(x,y) in loader_train:
    print (t)
    break

OSError: Traceback (most recent call last):
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 34, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-2-002f92b3a5d5>", line 21, in __getitem__
    depth=torch.from_numpy(h5py.File(self.data_dir+'/DEP/'+name,'r')['depthOut'].value).float()
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/h5py/_hl/files.py", line 271, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, swmr=swmr)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/h5py/_hl/files.py", line 101, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper (/tmp/pip-huypgcah-build/h5py/_objects.c:2840)
  File "h5py/_objects.p

# Loading Parameters

## Numpy File

In [17]:
pretrain = np.load('NYU_ResNet-UpProj.npy',encoding='latin1')
print(pretrain.item().keys())

dict_keys(['res3d_branch2c', 'bn2b_branch2c', 'layer2x_br2_BN', 'layer1', 'bn3b_branch2b', 'res2a_branch2c', 'res4a_branch1', 'res4e_branch2c', 'res3a_branch2b', 'layer16x_br1_ConvC', 'res5c_branch2c', 'layer8x_br1_BN', 'res2a_branch2a', 'res3c_branch2a', 'res5a_branch2b', 'res5b_branch2b', 'bn4b_branch2c', 'res3b_branch2a', 'bn2b_branch2b', 'res5b_branch2a', 'bn4a_branch2a', 'res2c_branch2c', 'res4a_branch2b', 'bn5a_branch2a', 'bn4f_branch2a', 'bn4a_branch2b', 'bn2a_branch2a', 'bn5c_branch2b', 'layer2x_br1_ConvB', 'bn4f_branch2c', 'bn4c_branch2c', 'layer8x_br1_ConvC', 'res4b_branch2a', 'layer4x_br1_BN', 'layer4x_BN', 'layer16x_br2_ConvA', 'layer16x_br2_ConvB', 'layer16x_br1_ConvD', 'bn2c_branch2c', 'res2c_branch2b', 'layer8x_Conv', 'bn4f_branch2b', 'res5c_branch2b', 'bn2b_branch2a', 'layer16x_br2_ConvC', 'bn4d_branch2b', 'layer8x_br2_ConvA', 'layer2x_br1_BN', 'layer16x_br1_ConvA', 'layer16x_br2_ConvD', 'bn4a_branch2c', 'ConvPred', 'bn3a_branch1', 'layer4x_br1_ConvA', 'bn4e_branch2c', 

In [29]:
pretrain.item()['layer4x_br1_ConvB']['weights'].shape


(2, 3, 512, 256)

## Mat File

In [6]:
import scipy.io

In [7]:
mat = scipy.io.loadmat('NYU_ResNet-UpProj.mat')

In [8]:
mat.keys()

dict_keys(['vars', '__globals__', 'meta', '__version__', 'layers', '__header__', 'params'])

In [9]:
mat['params'][0][0][1].shape

(7, 7, 3, 64)

In [ ]:
conv1_filter = mat['params'][0][0][1].transpose([3,2,0,1])

In [ ]:
conv1_filter.shape

In [ ]:
mat['params'][0][1][1].shape

In [ ]:
conv1_bias = mat['params'][0][1][1]

# Build Model

In [10]:
import torchvision.models as models

In [11]:
dtype = torch.cuda.FloatTensor

In [12]:
model = models.resnet50(pretrained=False)

In [13]:
mod = list(model.children())

In [14]:
mod

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True),
 ReLU (inplace),
 MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1)),
 Sequential (
   (0): Bottleneck (
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
     (relu): ReLU (inplace)
     (downsample): Sequential (
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
     )
   )
   (1): Bottleneck (
     (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1,

In [15]:
mod[0].weight.data.size()

torch.Size([64, 3, 7, 7])

In [ ]:
mod[0] = nn.Conv2d(3,64,7,stride=2, padding=3,bias=True)

In [ ]:
mod[0].weight.data = torch.from_numpy(conv1_filter)

In [ ]:
mod[0].bias.data = torch.from_numpy(conv1_bias)

In [ ]:
mod[1].__dict__.keys()

In [ ]:
mod[1]._parameters

In [ ]:
mod[4][0].conv1

In [ ]:
mod.pop()

In [ ]:
mod.pop()

In [ ]:
resnet50 = torch.nn.Sequential(*mod)

In [ ]:
del model

In [ ]:
class cropMore(nn.Module):
    def forward(self,x):
        N,C,H,W = x.size()
        return x[:,:,0:H-6,0:W-9].squeeze()

In [ ]:
class unpooling(nn.Module):
    def __init__(self,H,W):
        super(unpooling, self).__init__()
        self.indices = torch.zeros(H,W)
        for i in range(H):
            for j in range(W):
                self.indices[i,j] = i*2*2*W+j*2
        self.unpool = nn.MaxUnpool2d(2)
    def forward(self, x):
        N,C,H,W = x.size()
        indices = self.indices.expand(N,C,H,W)
        y = self.unpool(x, Variable(indices.type(torch.cuda.LongTensor),requires_grad=False))
        return y

In [ ]:
class upsampling(nn.Module):
    def __init__(self,C,H,W):
        super(upsampling, self).__init__()
        self.unpooling = unpooling(H,W)
        self.conv5 = nn.Conv2d(C,int(C/2),5,stride = 1,padding=2)
        self.ReLU = nn.ReLU()
        self.conv3 = nn.Conv2d(int(C/2),int(C/2),3,stride = 1,padding=1)
        self.conv5_2 = nn.Conv2d(C,int(C/2),5,stride = 1,padding=2)
    def forward(self, x):
        y0 = self.unpooling(x)
        y1 = self.conv5(y0)
        y1 = self.ReLU(y1)
        y1 = self.conv3(y1)
        y2 = self.conv5_2(y0)
        y3 = y1+y2
        y = self.ReLU(y3)
        return y

In [ ]:
upsample = torch.nn.Sequential(
    nn.Conv2d(2048,1024,1,stride = 1,padding=0),
    nn.BatchNorm2d(1024), #1024*10*8
    upsampling(1024,10,8),
    upsampling(512,20,16),
    upsampling(256,40,32),
    #upsampling(128,80,64),
    nn.Conv2d(128,1,3,stride = 1,padding=1),
    nn.ReLU(),
    cropMore()
    )
    

In [ ]:
resnet50.type(dtype)
upsample.type(dtype)

## Model Testing

In [ ]:
for t,(x,y) in enumerate(loader_train):
    x_var=Variable(x.type(dtype),requires_grad=False)
    #pred=new_model(x_var)
    x1 = resnet50(x_var)
    pred = upsample(x1)
    print(pred.size())
    break

In [ ]:
testModel = nn.MaxPool2d(2,2,return_indices = True)
testModel.type(dtype)

In [ ]:
x_var=Variable(torch.zeros(1,3,6,10).type(dtype),requires_grad=False)
#pred=new_model(x_var)
y,ind = testModel(x_var)
print(x_var.size())
print(y.size())
print(ind.size())
print(ind)


In [ ]:
testModel = unpooling(10,8)
testModel.type(dtype)

In [ ]:
x_var=Variable(torch.ones(1,4,10,8).type(dtype),requires_grad=False)
#pred=new_model(x_var)
y = testModel(x_var)
print(y)

## Training

In [ ]:
#images_var=Variable(images_pytorch.type(dtype),requires_grad=False)
#depths_var=Variable(depths_pytorch.type(dtype),requires_grad=False)

def loss_log(pred,y):
    ep = 1e-6
    N,W,H = pred.size()
    pred = pred.contiguous().view(N,-1)
    y = y.view(N,-1)
    y = y+ep
    d = pred - y.log()
    d[y <= 0] = 0
    n = W*H
    loss = (d.pow(2).sum(1) / n - 0.5 / n/n * d.sum(1).pow(2)).sum()
    loss /= N
    return loss
    


In [ ]:
def imshow_noax(img, normalize=True):
    """ Tiny helper to show images as uint8 and remove axis labels """

    if normalize:
        img_max, img_min = np.max(img), np.min(img)
        img = 255.0 * (img - img_min) / (img_max - img_min)

    plt.imshow(img.astype('uint8'))
    plt.gca().axis('off')

In [ ]:
for param in resnet50.parameters():
    param.requires_grad = False

In [ ]:
lr=1e-6
reg=1e-4
adam_optim2=optim.Adam(resnet50.parameters(),lr=lr,weight_decay=reg)

In [ ]:
lr=1e-5
reg=1e-4
adam_optim=optim.Adam(upsample.parameters(),lr=lr,weight_decay=reg)

In [ ]:
import pickle
print_every=50

def train(model, loss_fn, optimizer, num_epochs = 1, plot_every = 2):
    train_losses = []
    val_losses = []
    for epoch in range(num_epochs):
        model.train() # set the model to training mode, only effect batchnorm and dropout
        avg_train_loss=0
        num_batches=0
        for t,(x,y) in enumerate(loader_train):
            x_var=Variable(x.type(dtype),requires_grad=False)
            y_var=Variable(y.type(dtype),requires_grad=False)
            x1 = resnet50(x_var)
            pred = model(x1)
            loss = loss_fn(pred, y_var)
            #losses.append(loss.data.cpu().numpy())
            
            if (t+1) % print_every==0:
                print('t = %d, loss = %.4f' % (t+1, loss.data[0]))
            avg_train_loss+=loss.data[0]
            num_batches+=1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            del x,y
        avg_train_loss/=num_batches
        train_losses.append(avg_train_loss)
        
        num_batches=0
        avg_val_loss=0
        for t,(x,y) in enumerate(loader_val):
            x_var=Variable(x.type(dtype),requires_grad=False)
            y_var=Variable(y.type(dtype),requires_grad=False)
            x1 = resnet50(x_var)
            pred = model(x1)
            loss=loss_fn(pred,y_var)
            avg_val_loss+=loss.data[0]
            num_batches+=1
            del x,y
        avg_val_loss/=num_batches
        val_losses.append(avg_val_loss)
        print("epoch:",epoch,"average training loss: %.2f"%avg_train_loss,"validation loss: %.2f" %avg_val_loss)
        if(epoch % plot_every == 0):
            #with open('losses_ep'+ str(epoch)+ '.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
            #    pickle.dump([train_losses,val_losses], f)
            #torch.save(model.state_dict(), 'alldata_dict_ep'+str(epoch))
            plt.plot(train_losses)
            plt.plot(val_losses)
        
            

train(upsample,loss_log,adam_optim,num_epochs = 20)

In [ ]:
train(upsample,loss_log,adam_optim,num_epochs = 100)

In [ ]:
%matplotlib inline

def imshow_noax(img, normalize=True):
    """ Tiny helper to show images as uint8 and remove axis labels """

    if normalize:
        img_max, img_min = np.max(img), np.min(img)
        img = 255.0 * (img - img_min) / (img_max - img_min)

    plt.imshow(img.astype('uint8'))
    plt.gca().axis('off')

# visualize 5 images
NUM_SHOW=6
for t,(x,y) in enumerate(loader_train):
    x_var=Variable(x.type(dtype),requires_grad=False)
    y_var=Variable(y.type(dtype),requires_grad=False)
    upsample.train(False)
    x1 = resnet50(x_var)
    pred = upsample(x1)
    x_var=x_var.cpu()
    y_var=y_var.cpu()
    pred=pred.cpu()
    for i in range(NUM_SHOW):
        plt.subplot(3,NUM_SHOW,i+1)
        img=x_var[i,:,:,:].data.numpy()
        img=np.rollaxis(img,0,3)
        imshow_noax(img,False)
        plt.subplot(3,NUM_SHOW,NUM_SHOW+i+1)
        img=y_var[i,:,:].data.numpy()
        imshow_noax(img)
        plt.subplot(3,NUM_SHOW,2*NUM_SHOW+i+1)
        img=pred[i,:,:].data.numpy()
        img = np.exp(img)
        imshow_noax(img)
    break
plt.show()

In [ ]:
def evaluate(loader, model):
    ep = 1e-7

    thresh_1 = 0
    thresh_2 = 0
    abs_diff = 0
    rmse = 0

    for t,(x,y) in enumerate(loader):
        x_var=Variable(x.type(dtype),requires_grad=False)
        y_var=Variable(y.type(dtype),requires_grad=False)
        model.train(False)
        x1 = resnet50(x_var)
        pred = model(x1)

        #print(y_var.data.cpu().numpy())
        y_var=y_var.data.cpu().numpy()
        y_var = np.exp(y_var) + ep
        pred=pred.data.cpu().numpy() + ep
        num_var = np.shape(y_var)[0]*np.shape(y_var)[1]*np.shape(y_var)[2]

        # threshold
        thresh_mat = np.zeros_like(y_var)
        thresh_mat[np.maximum(y_var / pred, pred / y_var) < 1.25] = 1
        thresh_1 += np.sum(thresh_mat)
        thresh_2 += num_var
        #print('t1 = %d, t2 = %d', thresh_1, thresh_2)

        # relative absolute diffe
        abs_diff += np.sum(np.absolute(y_var - pred) / y_var) / num_var

        rmse += np.sqrt(np.sum((y_var - pred) * (y_var - pred)) / num_var)
        


    abs_diff /= len(loader)
    rmse /= len(loader)

    print('percentage within threshold: ', thresh_1 / thresh_2)
    print('relative absolute diff = ', abs_diff)
    print('rmse = ', rmse)

In [ ]:
evaluate(loader_val, upsample)

## Old

In [ ]:

def train(model, loss_fn, optimizer, optimizer2, num_epochs = 1, plot_every = 10):
    losses = []
    for epoch in range(num_epochs):
        model.train() # set the model to training mode, only effect batchnorm and dropout
        avg_train_loss=0
        num_batches=0
        for t,(x,y) in enumerate(loader_train):
            x_var=Variable(x.type(dtype),requires_grad=False)
            y_var=Variable(y.type(dtype),requires_grad=False)
            x1 = resnet50(x_var)
            pred = model(x1)
            loss = loss_fn(pred, y_var)
            losses.append(loss.data.cpu().numpy())
            
            if (t+1) % print_every==0:
                print('t = %d, loss = %.4f' % (t+1, loss.data[0]))
            avg_train_loss+=loss.data[0]
            num_batches+=1
            optimizer.zero_grad()
            optimizer2.zero_grad()
            loss.backward()
            optimizer.step()
            optimizer2.step()
            del x,y
        avg_train_loss/=num_batches
        num_batches=0
        avg_val_loss=0
        for t,(x,y) in enumerate(loader_val):
            x_var=Variable(x.type(dtype),requires_grad=False)
            y_var=Variable(y.type(dtype),requires_grad=False)
            x1 = resnet50(x_var)
            pred = model(x1)
            loss=loss_fn(pred,y_var)
            avg_val_loss+=loss.data[0]
            num_batches+=1
            del x,y
        avg_val_loss/=num_batches
        print("epoch:",epoch,"average training loss: %.2f"%avg_train_loss,"validation loss: %.2f" %avg_val_loss)
        if(epoch % plot_every == 0):
            plt.plot(losses)
            

train(upsample,log_loss,adam_optim,adam_optim2,num_epochs = 20)

In [ ]:
del upsample
del resnet50
del model

In [ ]:
%matplotlib inline

def imshow_noax(img, normalize=True):
    """ Tiny helper to show images as uint8 and remove axis labels """

    if normalize:
        img_max, img_min = np.max(img), np.min(img)
        img = 255.0 * (img - img_min) / (img_max - img_min)

    plt.imshow(img.astype('uint8'))
    plt.gca().axis('off')

# visualize 5 images
NUM_SHOW=5
for t,(x,y) in enumerate(loader_train):
    x_var=Variable(x.type(dtype),requires_grad=False)
    y_var=Variable(y.type(dtype),requires_grad=False)
    upsample.train(False)
    x1 = resnet50(x_var)
    pred = upsample(x1)
    x_var=x_var.cpu()
    y_var=y_var.cpu()
    pred=pred.cpu()
    for i in range(NUM_SHOW):
        plt.subplot(3,NUM_SHOW,i+1)
        img=x_var[i,:,:,:].data.numpy()
        img=np.rollaxis(img,0,3)
        imshow_noax(img,False)
        plt.subplot(3,NUM_SHOW,NUM_SHOW+i+1)
        img=y_var[i,:,:].data.numpy()
        imshow_noax(img)
        plt.subplot(3,NUM_SHOW,2*NUM_SHOW+i+1)
        img=pred[i,:,:].data.numpy()
        img = np.exp(img)
        imshow_noax(img)
    break
plt.show()

In [ ]:
train(upsample,log_loss,adam_optim,adam_optim2,num_epochs = 20)

In [ ]:
NUM_SHOW=5
for t,(x,y) in enumerate(loader_train):
    x_var=Variable(x.type(dtype),requires_grad=False)
    y_var=Variable(y.type(dtype),requires_grad=False)
    upsample.train(False)
    x1 = resnet50(x_var)
    pred = upsample(x1)
    x_var=x_var.cpu()
    y_var=y_var.cpu()
    pred=pred.cpu()
    for i in range(NUM_SHOW):
        plt.subplot(3,NUM_SHOW,i+1)
        img=x_var[i,:,:,:].data.numpy()
        img=np.rollaxis(img,0,3)
        imshow_noax(img,False)
        plt.subplot(3,NUM_SHOW,NUM_SHOW+i+1)
        img=y_var[i,:,:].data.numpy()
        imshow_noax(img)
        plt.subplot(3,NUM_SHOW,2*NUM_SHOW+i+1)
        img=pred[i,:,:].data.numpy()
        img = np.exp(img)
        imshow_noax(img)
    break
plt.show()

In [ ]:
def evaluate(loader):
    ep = 1e-7

    thresh_1 = 0
    thresh_2 = 0
    abs_diff = 0
    rmse = 0

    for t,(x,y) in enumerate(loader):
        x_var=Variable(x.type(dtype),requires_grad=False)
        y_var=Variable(y.type(dtype),requires_grad=False)
        coarse_model.train(False)
        pred=coarse_model(x_var)

        #print(y_var.data.cpu().numpy())
        y_var=y_var.data.cpu().numpy() + ep
        pred=pred.data.cpu().numpy() + ep
        num_var = np.shape(y_var)[0]*np.shape(y_var)[1]*np.shape(y_var)[2]

        # threshold
        thresh_mat = np.zeros_like(y_var)
        thresh_mat[np.maximum(y_var / pred, pred / y_var) < 1.25] = 1
        thresh_1 += np.sum(thresh_mat)
        thresh_2 += num_var
        #print('t1 = %d, t2 = %d', thresh_1, thresh_2)

        # relative absolute diffe
        abs_diff += np.sum(np.absolute(y_var - pred) / y_var) / num_var

        rmse += np.sqrt(np.sum((y_var - pred) * (y_var - pred)) / num_var)


    abs_diff /= len(loader_train)

    print('percentage within threshold: ', thresh_1 / thresh_2)
    print('relative absolute diff = ', abs_diff)
    print('rmse = ', rmse)

In [ ]:
evaluate(loader_val)

In [ ]:

def train_overfit(model, loss_fn, optimizer, num_epochs = 1, plot_every = 10):
    losses = []
    for epoch in range(num_epochs):
        model.train() # set the model to training mode, only effect batchnorm and dropout
        avg_train_loss=0
        num_batches=0
        for t,(x,y) in enumerate(loader_train):
            x_var=Variable(x.type(dtype),requires_grad=False)
            y_var=Variable(y.type(dtype),requires_grad=False)
            pred=model(x_var)
            loss = loss_fn(pred, y_var)
            losses.append(loss.data.cpu().numpy())
            
            if (t+1) % print_every==0:
                print('t = %d, loss = %.4f' % (t+1, loss.data[0]))
            avg_train_loss+=loss.data[0]
            num_batches+=1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            break
        avg_train_loss/=num_batches
        num_batches=0

        print("epoch:",epoch,"average training loss: %.2f"%avg_train_loss)
        if(epoch % plot_every == 0):
            plt.plot(losses)
            

train_overfit(coarse_model,loss_log,adam_optim,40)

In [ ]:
NUM_SHOW=5
for t,(x,y) in enumerate(loader_train):
    x_var=Variable(x.type(dtype),requires_grad=False)
    y_var=Variable(y.type(dtype),requires_grad=False)
    coarse_model.train(False)
    pred=coarse_model(x_var)
    x_var=x_var.cpu()
    y_var=y_var.cpu()
    pred=pred.cpu()
    for i in range(NUM_SHOW):
        plt.subplot(3,NUM_SHOW,i+1)
        img=x_var[i,:,:,:].data.numpy()
        img=np.rollaxis(img,0,3)
        imshow_noax(img,False)
        plt.subplot(3,NUM_SHOW,NUM_SHOW+i+1)
        img=y_var[i,:,:].data.numpy()
        imshow_noax(img)
        plt.subplot(3,NUM_SHOW,2*NUM_SHOW+i+1)
        img=pred[i,:,:].data.numpy()
        img = np.exp(img)
        imshow_noax(img)
    break
plt.show()

In [ ]:
%matplotlib inline
a = [1,2,3,4]#np.array([1,2,3,4])
a.append(5)
plt.plot(a)

In [ ]:
print(loader_train)

In [ ]:
a = torch.zeros(3,3)
a[0,1] = 2
a[0,2] = 4
b = torch.sum(a,0)

print(a)
print(b)

In [ ]:
class UnConv(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        channel1 = nn.Sequential(
            nn.MaxUnpool2d(kernel_size=2, stride=None, padding=0),
            nn.Conv2d(C,int(C/2),5,stride = 1,padding=2),
            nn.ReLU()
            )
        channel1.type(dtype)
        return channel1(x)